# Mouse olfactory bulb

Thor reveals cell-resolution mouse olfactory bulb structure.

In this tutorial, we show inference of cell-level spatial gene expression from the Visium spot-level spatial transcriptome and a whole slide image (H&E staining) using Thor. 

The spatial transcriptome dataset is the Adult Mouse Olfactory Bulb. 
The input data can be downloaded from our [google drive](https://drive.google.com/drive/folders/1CaUHMRUcKT-qGY2_lMkOcVVu_7OW50S-?usp=sharing) or 10x Genomics [website](https://www.10xgenomics.com/resources/datasets/adult-mouse-olfactory-bulb-1-standard-1). 

For installation of Thor, please refer to [this installation guide](../installation.rst).

## Import the packages

In [ ]:
import sys
import os
import logging
import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(name)s - %(levelname)s - %(message)s')

now = datetime.datetime.now()
logger.info(f"Current Time: {now}")

In [ ]:
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

import warnings

warnings.filterwarnings("ignore")
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=300)
sc.settings.verbosity = 'error'


from thor.pp import WholeSlideImage, Spatial
from thor.finest import fineST
from thor.pl import colors 

## Preprocessing

### Preprocessing the whole slide image, including cell segmentation and morphological feature extraction. If not specified, the segmentation data will be saved in a directory (created by Thor) WSI_{sn}. 

The WSI can be downloaded directly from [google drive](https://drive.google.com/drive/folders/1CaUHMRUcKT-qGY2_lMkOcVVu_7OW50S-?usp=sharing).

The outcomes of image preprocessing are two files:
- cell mask
- cell features

In [3]:
sn = 'MOB_10x'
image_path = "Visium_Mouse_Olfactory_Bulb_image.tif"

wsi = WholeSlideImage(image_path, name=sn)

Here we use Cellpose to segment the nuclei. Notice the segmentation can take a long time without GPU. The output file is a `n_pixel_row` x `n_pixel_col` matrix. We'll skip this step and load the pre-computed segmentation results from [here]https://drive.google.com/drive/folders/1CaUHMRUcKT-qGY2_lMkOcVVu_7OW50S-?usp=sharing). In the downloaded folder, both the cell segmentation and cell features files are included.

In [ ]:
%%script echo "Skip cell segmentation"

wsi.process(method='cellpose')

### Preprocessing spatial transcriptome. We used SCANPY to generate an adata for the spots including QC.

We follow the standarized steps used by SCANPY to create the spot adata from the space ranger output directory (`visium_dir`). The spot adata contains the expression matrix, the location of the spots as pixel positions on the WSI, and the hires, lowres images with scalefactors.
- spot.h5ad

You can skip this step and download the files [here](https://drive.google.com/drive/folders/1CaUHMRUcKT-qGY2_lMkOcVVu_7OW50S-?usp=sharing)

In [ ]:
st_dir = "10xOlfactory"

trans = Spatial(sn, st_dir)
trans.process_transcriptome(perform_QC=False)

Basic transcriptome preprocessing

In [6]:
spot_adata_path = trans.spot_adata_path
adata_spot = sc.read_h5ad(spot_adata_path)
sc.pp.normalize_total(adata_spot, target_sum=1e4)
sc.pp.log1p(adata_spot)

adata_spot.write_h5ad(spot_adata_path)

## Predicting cell-level gene expression using Markov graph diffusion 

After finishing the preprocessing, you should have those files:
- The original WSI (`image_path`)
- The cell(nuclei) mask and features (in directory "./WSI_MOB_10x")
- The spot-level gene expression (in directory "./Spatial_MOB_10x")


In [7]:
outdir = os.getcwd()

image_process_dir = os.path.join(outdir, "WSI_MOB_10x")
cell_mask_path = os.path.join(image_process_dir, "cell_mask.npz")
cell_feature_path = os.path.join(image_process_dir, "cell_features.csv")

The first step is to map the spot gene expression to the segmented cells. We use the nearest neighbors approach. This cell-level gene expression is the starting point for the diffusion process.

In [ ]:
MOB = fineST(
    image_path,
    name=sn,
    spot_adata_path=spot_adata_path,
    cell_features_csv_path=cell_feature_path,
    cell_features_list=['x', 'y', 'mean_gray', 'std_gray', 'entropy_img', 'mean_r', 'mean_g', 'mean_b', 'std_r', 'std_g', 'std_b'],
)

MOB.prepare_input(mapping_margin=2)

In [ ]:
MOB.adata

Second step is set up genes for prediction. For the sake of time, we'll show prediction of a few genes. The same Markov transition matrix can be applied to all genes. The user-defined genes can be input by either in a 1-column text file or directly as the attribute `genes` of the `MOB` object

In [ ]:
MOB.genes = ['Eomes', 'Uchl1', 'Pcp4l1', 'Col6a1', 'Tbx21', 'Slc17a6', 'Bmp7', 'Camk2a', 'Cystm1', 'Penk', 'Gria1', 'Nap1l5', 'Serpine2', 'Kctd12', 'Pde5a', 'Syt7', 'Vtn']
MOB.set_genes_for_prediction(genes_selection_key=None)
MOB.adata.var[MOB.adata.var['used_for_prediction']]

In this case study, we include the effect of input spatial transcriptome in constructing the cell-cell network.

- Burn in 5 steps using only histology features (vanilla version) for all highly variable genes. 
- Load the burnin result, perform PCA, and use the transcriptome to adjust the cell-cell network. 
- The cell-cell network with transcriptome PCA information will be used to perform the production diffusion.

Run time (~ 3 mins on MacbookPro M2)

In [ ]:
MOB.recipe = 'gene'
MOB.set_params(
    is_rawCount=False,
    out_prefix="fineST",
    write_freq=40,
    n_iter=40,
    conn_csr_matrix="force",
    smoothing_scale=0.8,
    node_features_obs_list=['spot_heterogeneity'],
    n_neighbors=5,
    geom_morph_ratio=0.5,
    geom_constraint=0,
    inflation_percentage=1,
    regulate_expression_mean=False,
    stochastic_expression_neighbors_level='spot',
    smooth_predicted_expression_steps=10,
    reduced_dimension_transcriptome_obsm_key='X_pca',
    adjust_cell_network_by_transcriptome_scale=1,
    n_jobs=4)

MOB.predict_gene_expression()

### Check the expression of a few genes.

In [ ]:
ad_thor = MOB.load_result('fineST_40_samp10.npz')
ad_thor

In [ ]:
genes = ['Eomes', 'Uchl1', 'Pcp4l1', 'Col6a1', 'Tbx21', 'Slc17a6', 'Bmp7']

ad_spot = sc.read_h5ad(spot_adata_path)
sc.pl.spatial(ad_spot, color=genes, vmax='p99', frameon=False, ncols=4)

In [ ]:
genes = ['Eomes', 'Uchl1', 'Pcp4l1']

sc.set_figure_params(scanpy=True, dpi=80, dpi_save=100, facecolor='k', frameon=False)
fw2 = LinearSegmentedColormap.from_list('fw2', colors.continuous_palettes['fireworks2'])

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
i = 0

for gene in genes:
    sc.pl.spatial(ad_thor, color=gene, spot_size=15, vmin='p5', vmax='p95', colorbar_loc=None, cmap='coolwarm', img_key=None, show=False, ax=axes[i])
    axes[i].set_title(gene, color='w')
    i += 1
plt.show()

Visualize module-specific genes

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from thor.pl import colors 

mod_genes = ['Col6a1', 'Tbx21', 'Slc17a6', 'Bmp7']


sc.set_figure_params(scanpy=True, dpi=80, dpi_save=100, facecolor='k', frameon=False)
yb = LinearSegmentedColormap.from_list('yb', colors.continuous_palettes['blueYellow'])

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
i = 0

for gene in mod_genes:
    sc.pl.spatial(ad_thor, color=gene, spot_size=15, vmin='p2', vmax='p98', colorbar_loc=None, cmap=yb, img_key=None, show=False, ax=axes[i])
    axes[i].set_title(gene, color='w')
    i += 1
plt.show()